## Part 1: manipulating YAML files with OmegaConf

See the full documentation here: [https://omegaconf.readthedocs.io/en/latest/usage.html#access-and-manipulation](https://omegaconf.readthedocs.io/en/latest/usage.html#access-and-manipulation)

In [1]:
from omegaconf import OmegaConf

In [2]:
conf = OmegaConf.load('configs/config.yaml')

In [3]:
print(type(conf))

<class 'omegaconf.dictconfig.DictConfig'>


In [4]:
# Accessing values
print(conf.dataset.image.channels)
print(conf['dataset']['classes'])

3
10


In [5]:
# Modifying a value
conf.dataset.classes = 15

## Part 2: Using Hydra

In [6]:
%run -i main.py

dataset:
  image:
    size: 124
    channels: 3
  classes: 10
feature_extractor:
  _target_: torchvision.models.alexnet
  pretrained: true
classifier:
  dense1:
    _target_: torch.nn.Linear
    in_features: 9216
    out_features: 100
  dense2:
    _target_: torch.nn.Linear
    in_features: 100
    out_features: 10

[2021-05-24 09:33:30,309][__main__][INFO] - Working dir: d:\Codice\git-repositories\hydra-tutorial\outputs\2021-05-24\09-33-30


In [7]:
# Modify a flag
%run -i main.py dataset.classes=15

dataset:
  image:
    size: 124
    channels: 3
  classes: 15
feature_extractor:
  _target_: torchvision.models.alexnet
  pretrained: true
classifier:
  dense1:
    _target_: torch.nn.Linear
    in_features: 9216
    out_features: 100
  dense2:
    _target_: torch.nn.Linear
    in_features: 100
    out_features: 15

[2021-05-24 09:33:32,405][__main__][INFO] - Working dir: d:\Codice\git-repositories\hydra-tutorial\outputs\2021-05-24\09-33-32


In [8]:
# Add a flag
%run -i main.py +custom_param=True

dataset:
  image:
    size: 124
    channels: 3
  classes: 10
feature_extractor:
  _target_: torchvision.models.alexnet
  pretrained: true
classifier:
  dense1:
    _target_: torch.nn.Linear
    in_features: 9216
    out_features: 100
  dense2:
    _target_: torch.nn.Linear
    in_features: 100
    out_features: 10
custom_param: true

[2021-05-24 09:33:32,702][__main__][INFO] - Working dir: d:\Codice\git-repositories\hydra-tutorial\outputs\2021-05-24\09-33-32


In [9]:
# Change the working directory
%run -i main.py hydra.run.dir='outputs/custom_folder'

dataset:
  image:
    size: 124
    channels: 3
  classes: 10
feature_extractor:
  _target_: torchvision.models.alexnet
  pretrained: true
classifier:
  dense1:
    _target_: torch.nn.Linear
    in_features: 9216
    out_features: 100
  dense2:
    _target_: torch.nn.Linear
    in_features: 100
    out_features: 10

[2021-05-24 09:33:34,063][__main__][INFO] - Working dir: d:\Codice\git-repositories\hydra-tutorial\outputs\custom_folder


## Part 3: Instantiating a configuration file manually

In [10]:
from hydra import initialize, compose

In [11]:
# Initialize Hydra
initialize('configs')

hydra.initialize()

In [12]:
cfg = compose(config_name='config')

## Part 4: Instantiating an object

In [13]:
import torch
from collections import OrderedDict
from hydra.utils import instantiate

In [14]:
net = instantiate(cfg.feature_extractor)

In [15]:
print(instantiate(cfg.classifier))

{'dense1': Linear(in_features=9216, out_features=100, bias=True), 'dense2': Linear(in_features=100, out_features=10, bias=True)}


In [16]:
# Printing the configuration with resolution
print(OmegaConf.to_yaml(cfg.classifier, resolve=True))

dense1:
  _target_: torch.nn.Linear
  in_features: 9216
  out_features: 100
dense2:
  _target_: torch.nn.Linear
  in_features: 100
  out_features: 10



In [ ]:
classifier = torch.nn.Sequential(
    OrderedDict(instantiate(cfg.classifier)
)

In [ ]:
net.classifier = classifier

In [ ]:
# Test it works
xb = torch.randn((8, cfg.dataset.image.channels, cfg.dataset.image.size, cfg.dataset.image.size))
print(net(xb).shape)

In [ ]:
# Instantiate the large model
cfg = compose(config_name='config', overrides=['classifier=large'])

## Part 5: An example of multiple run

In [ ]:
# We need to reset HydraGlobal because we called initialize before
hydra.core.global_hydra.GlobalHydra.instance().clear()

In [ ]:
%run -i main.py -m classifier=small,large

## Part 6: Validating the configuration

In [ ]:
# Try overriding a parameter with an invalid type
%run -i main.py dataset.classes=0.5